In [26]:
#!/usr/bin/env python
# coding: utf-8
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
import gc
import decoupler as dc
data_type = 'float32'

In [29]:
# Function to import a single slide
def read_and_qc(sample_name, module_table, module_folder, path=sp_data_folder, force_filter = True):
    r""" This function reads the data for one 10X spatial experiment into the anndata object.
    It also calculates QC metrics. Modify this function if required by your workflow.

    :param sample_name: Name of the sample
    :param path: path to data
    """

    adata = sc.read_visium(path + str(sample_name) + '/outs',
                           count_file='filtered_feature_bc_matrix.h5', 
                           load_images=True)
    
    adata.obs['sample'] = sample_name
    adata.var['SYMBOL'] = adata.var_names

    # Calculate QC metrics
    sc.pp.calculate_qc_metrics(adata, inplace=True)
    adata.var['mt'] = [gene.startswith('MT-') for gene in adata.var['SYMBOL']]
    adata.var['rps'] = [gene.startswith('RPS') for gene in adata.var['SYMBOL']]
    adata.var['mrp'] = [gene.startswith('MRP') for gene in adata.var['SYMBOL']]
    adata.var['rpl'] = [gene.startswith('RPL') for gene in adata.var['SYMBOL']]
    adata.obs['mt_frac'] = adata[:,adata.var['mt'].tolist()].X.sum(1).A.squeeze()/adata.obs['total_counts']

    # add sample name to obs names
    adata.obs["sample"] = [str(i) for i in adata.obs['sample']]
    adata.obs_names = adata.obs["sample"] + '_' + adata.obs_names
    adata.obs.index.name = 'spot_id'
    adata.var["duplicated"] = adata.var['SYMBOL'].duplicated(keep = "first")
    adata = adata[:, ~adata.var['duplicated'].values]
    
    if force_filter:
        # First filter: mt and rb genes
        # mitochondria-encoded (MT) genes should be removed for spatial mapping
        adata.obsm['mt'] = adata[:,   adata.var['mt'].values | 
                              adata.var['rps'].values |
                              adata.var['mrp'].values |
                              adata.var['rpl'].values].X.toarray() 
        
        adata = adata[:, ~ (adata.var['mt'].values | 
                              adata.var['rps'].values |
                              adata.var['mrp'].values |
                              adata.var['rpl'].values)]
        
        # Second filter
        # Genes expressed in less than 10 spots
        adata = adata[:, adata.var['n_cells_by_counts'].values > 10]
        
        # Third filter
        # spots with no information (less than 300 genes and 500 UMIs)
        sc.pp.calculate_qc_metrics(adata, inplace=True)
        adata = adata[(adata.obs['n_genes_by_counts'].values > 300) & 
              (adata.obs['total_counts'].values > 500), :]
    
    
    sc.pp.normalize_total(adata, inplace=True)
    sc.pp.log1p(adata)
    
    dc.run_wmean(adata, net = module_table, source = "module", 
             target = "feature", weight = "weight", use_raw=False)
    
    csv_out = module_folder + "decoupler/" + str(sample_name) + ".csv"
    
    adata.obsm["wmean_norm"].to_csv(csv_out)
    
    return adata

In [30]:
# Set paths to data and results used through the document:
sp_data_folder = '/Volumes/RicoData2/MI_project/MI_revisions/visium_data/'
samples = [f for f in os.listdir(sp_data_folder) if f != ".DS_Store"]

In [31]:
factor_folder = "/Users/ricardoramirez/Dropbox/PhD/Research/MOFAcell/results/MOFA_mcell/factor_desc/"

In [33]:
module_folder = factor_folder + "Factor2_char/"
module_file = module_folder + "gene_modules.csv"
module_table = pd.read_csv(module_file)
module_table["weight"] = 1

In [42]:
for visium_s in samples:
    v_adata = read_and_qc(sample_name = visium_s, 
                          module_table = module_table, 
                          module_folder = module_folder)

/Users/ricardoramirez/opt/miniconda3/envs/sc_py/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/ricardoramirez/opt/miniconda3/envs/sc_py/lib/python3.9/site-packages/scanpy/preprocessing/_qc.py:135: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[obs_metrics.columns] = obs_metrics
/Users/ricardoramirez/opt/miniconda3/envs/sc_py/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/Users/ricardoramirez/opt/miniconda3/envs/sc_py/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/ricardoramirez/opt/miniconda3/envs/sc_py/lib/python3.

In [43]:
module_folder = factor_folder + "Factor3_char/"
module_file = module_folder + "gene_modules.csv"
module_table = pd.read_csv(module_file)
module_table["weight"] = 1

In [44]:
for visium_s in samples:
    v_adata = read_and_qc(sample_name = visium_s, 
                          module_table = module_table, 
                          module_folder = module_folder)

/Users/ricardoramirez/opt/miniconda3/envs/sc_py/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/ricardoramirez/opt/miniconda3/envs/sc_py/lib/python3.9/site-packages/scanpy/preprocessing/_qc.py:135: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[obs_metrics.columns] = obs_metrics
/Users/ricardoramirez/opt/miniconda3/envs/sc_py/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/Users/ricardoramirez/opt/miniconda3/envs/sc_py/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/ricardoramirez/opt/miniconda3/envs/sc_py/lib/python3.